In [2]:
import pandas as pd
import duckdb

import mlflow


In [18]:
# Caminho do banco
db_path = "../../../data/duckdb/database.duckdb"

# Conexão com o banco DuckDB
con = duckdb.connect(db_path)

# Define raiz do projeto (ajuste conforme seu ambiente)
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Carrega os dados de feature engineering
df = con.execute("SELECT * FROM feature.clusterizacao_cliente").df()

inference_df = df.drop('client_id', axis=1).to_numpy()

# Inferência

In [6]:
# Define experimento
mlflow.set_experiment("clusterizacao_cliente")

model_name = "clusterizacao_cliente"

champion_model = mlflow.pyfunc.load_model(f"models:/{model_name}@champion")

In [7]:
y_pred = champion_model.predict(inference_df)

In [9]:
df['cluster'] = y_pred

df = df[['client_id', 'cluster']]

# Salvar resultados

In [12]:
con.execute("""
CREATE TABLE IF NOT EXISTS output.clusterizacao_cliente (
    client_id VARCHAR,
    cluster VARCHAR
)
""")

In [13]:
# Limpa dados se as tabelas já existirem
con.execute("DELETE FROM output.clusterizacao_cliente")

In [14]:
# Registra como tabelas temporárias
con.register("df", df)

In [15]:
# Insere os dados nas tabelas gold
con.execute("INSERT INTO output.clusterizacao_cliente SELECT * FROM df")

In [16]:
con.close()